## 세티즌 데이터 확인용
#### 스마트폰 크롤링 - 이름, 저장공간, 제품코드, 통신사, 설명, 상태, 구성품, 가격, 시간, url, 판매여부
##### 0328 

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

In [2]:
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 \
(KHTML, like Gecko) Chrome/99.0.4844.83 Safari/537.36'}

smartphone = []
name = []
storage = []
product_code = []
memo = []
condition = []
price = []
upload_time = []
sold = []
url_info = []
site = []
category = []



#마지막 페이지까지 크롤링 반복
for i in range(1,500):
    url = f"https://market.cetizen.com/market.php?p=1&keyword_p=&pno=&just_one=&just_one_name=&product_no=&pw=&pm=&m_no=&p_title=&pname=&p_key=&p_modelno=&p_model=&storage=&p_cat=&pm_modal=&series_modal=&datamode=&q=&at=&section=&cat%5B0%5D=3&price_range=0%3B5000000&auc_price1=0&auc_price2=5%2C000%2C000&storage_range=0%3B1024&storage1=0&storage2=1024&escrow_motion=3&select_arrange=headnum&desc=+&remove_dup=&list_type=1&p=400&p=401&p=411&p=421&p=431&p={i}"
    r = requests.get(url, headers=headers)
    r.status_code

    soup = BeautifulSoup(r.text,'html.parser')


    # 한페이지 정보 가져와 smartphone에 저장
    smartphone = soup.find_all('div', class_= 'col-10')
    smartphone = smartphone[0::2]

    
    # 각 정보 해당 리스트에 저장하기
    for info in smartphone:
        
        # 05/25 전 데이터 수집
        time_info = info.find("span", class_='gray2').text
        if(time_info == '05-25'): break
        
        # 상품 이름 
        name_info = info.find("span", class_='fsize6').text
        name.append(name_info)

        # 저장공간 정보는 정보 유무 판단 후, 정보 없는 경우 '정보없음'으로 저장
        if( info.find("span", class_='bg-gray3') ):
            storage.append(info.find("span", class_='bg-gray3').text)
        else : storage.append("정보없음")
    
        # 상품 코드 
        code_info = info.find("span", class_='ls-0').text
        product_code.append(code_info)
        
        # 상품 설명 
        memo_info = info.find("span", class_='pointer').text
        memo.append(memo_info)
        
        # 상품 상태 
        condition_info = info.find(['div','span'], class_= ['box_noicon_red', 'box_noicon_on']).text
        condition.append(condition_info)

        # 가격 정보는 삭제된(==거래완료된) 정보일 경우와 아닌 경우로 구분하여 저장
        if ( info.select("span > del") ) : 
            price.append(info.find("del").text)
        elif ( info.find("span", class_='black0')) : 
            price.append(info.find("span", class_='black0').text)
        else : 
            price.append( info.find_all("span", class_='gray1')[1].text )
        
        # 업로드 시간
        time_info = info.find("span", class_='gray2').text
        if(':' in time_info) : 
            upload_time.append('06-02')
        elif('전' in time_info) :
            upload_time.append('06-02')
        else : upload_time.append(time_info)
        
        # 거래 완료 
        if ( info.find("span" , class_= "ml-2")) : 
            sold.append("거래완료")
        else : sold.append("-")
            
        # url
        url_add = 'https://market.cetizen.com/' + info.find('a')['href']
        url_info.append(url_add)
            
        # 사이트명과 카테고리
        site.append("cetizen")
        category.append("휴대폰(스마트폰)")

        
    time.sleep(1)


In [3]:
# DataFrame 생성 및 확인

df = pd.DataFrame(
        {"사이트" : site, 
         "카테고리" : category,
         "판매여부" : sold,
         "이름" : name,
         "저장공간" : storage,
         "제품코드" : product_code,
         "상태" : condition,
         "가격" : price,
         "업로드시간" : upload_time,
         "설명" : memo,
         "url" : url_info
        }
    )

df.shape
df


,사이트,카테고리,판매여부,이름,저장공간,제품코드,상태,가격,업로드시간,설명,url
0,cetizen,휴대폰(스마트폰),-,갤럭시S22,256GB,SM-S901N,새제품,"799,000",06-02,갤럭시S22 그린 자급제 완박(미개봉),https://market.cetizen.com//market.php?q=view&...
1,cetizen,휴대폰(스마트폰),-,갤럭시퀀텀2,128GB,SM-A826S,새제품,"420,000",06-02,A826 흰색 22년4월 새제품 공기계 풀박스 3사와 알뜰폰전통신사 사용가능,https://market.cetizen.com//market.php?q=view&...
2,cetizen,휴대폰(스마트폰),-,갤럭시퀀텀3,128GB,SM-M536S,새제품,"450,000",06-02,M536 퀀텀3 검정 22년4월 새제품 공기계 풀박스 3사와 알뜰폰전통신사 사용가능...,https://market.cetizen.com//market.php?q=view&...
3,cetizen,휴대폰(스마트폰),-,갤럭시퀀텀3,128GB,SM-M536S,새제품,"450,000",06-02,M536 퀀텀3 블루 22년4월 새제품 공기계 풀박스 3사와 알뜰폰전통신사 사용가능...,https://market.cetizen.com//market.php?q=view&...
4,cetizen,휴대폰(스마트폰),-,갤럭시 폴더2 2021,32GB,SM-G160N 2021,새제품,"159,000",06-02,G160 2021 레드 22년4월 새제품 공기계 풀박스 3사와 알뜰폰전통신사 사용가능,https://market.cetizen.com//market.php?q=view&...
...,...,...,...,...,...,...,...,...,...,...,...
12070,cetizen,휴대폰(스마트폰),거래완료,갤럭시 노트8,64GB,SM-N950S,중급,"169,000",05-19,노트8 aaaa급!!,https://market.cetizen.com//market.php?q=view&...
12071,cetizen,휴대폰(스마트폰),거래완료,갤럭시 S20,128GB,SM-G981N,하급,"200,000",05-19,S20 화이트,https://market.cetizen.com//market.php?q=view&...
12072,cetizen,휴대폰(스마트폰),거래완료,V50s ThinQ,256GB,LM-V510N,중급,"137,000",05-19,V50S(V510) 256GB,https://market.cetizen.com//market.php?q=view&...
12073,cetizen,휴대폰(스마트폰),거래완료,아이폰11,64GB,A2221,중급,"370,000",05-19,아이폰 11 64gb 배터리 성능 87%,https://market.cetizen.com//market.php?q=view&...


In [4]:
# df.to_csv('cetizendata_smartphone', index=False)
df.to_excel("cetizendata_smartphone_0602.xlsx")